In [1]:
from math import atan, exp, sqrt, log
import random

In [2]:
SIZE = 8

In [3]:
def _pretty_print(m):
#   top_border = '┍' + ('─' * (len(m) * 2 - 2)) + '┑\n'
#   bottom_border = '\n┕' + ('─' * (len(m) * 2 - 2)) + '┙'
  rows = ['│'.join(['{:1}'.format('•' if item == 1 else '╌') for item in row])
                   for row in m]
  board = '│\n'.join(rows)
  print(board + '│')

In [4]:
# Given a queen in a board, return how many others queens attack her
class GameNode():
  def __init__(self, board=[], position=[]):
    self.n = len(board) if board else len(position) # board dimension
    if position or not board:
      self.position = position
    else:
      self.position = [0 for r in board]

      for j in range(self.n): # loop into columns
        for i in range(self.n):
          if board[i][j] == 1:
            # set the row of the queen in the current column
            self.position[j] = i
            break

    self.cost = None
    self.weight = None
    self.max_weight = sum(list(range(self.n)))
    self.children = []
    
  def generate_board(self):
    board = []

    for i in range(self.n):
      board.append([0 for i in range(self.n)])

    for j in range(self.n):
      board[self.position[j]][j] = 1
    
    return board

  def count_attacks(self, j):
    n = self.n

    i = self.position[j]

    d1 = i - j
    d2 = i + j

    attacks = 0

    for c in range(n): # loop into columns
      if c != j: # if the column is different of the given
        # If a queen exists in the same row
        # or in the main diagonal (\)
        # or in the secondary diagonal (/)
        if ((self.position[c] == i) or
          (d1 < n and d1 >= 0 and self.position[c] == d1) or 
          (d2 < n and d2 >= 0 and self.position[c] == d2)):
          attacks += 1

      d1 += 1
      d2 -= 1

    return attacks

  # Given a board, return the its total cost
  def state_cost(self):
    cost = 0
    for j in range(self.n): # loop into columns
      cost += self.count_attacks(j)

    # Due the attacks are counted two times for any pair of queens
    # the total sum is divided by 2
    self.cost = int(cost / 2)
    
    if self.cost == 0:
      self.weight = float("inf")
    else:
      self.weight = self.n / cost

  def generate_random_child(self):
    n = self.n

    j = random.randint(0, n - 1)
    i = random.randint(0, n - 1)
    
    while self.position[j] == i:
      i = random.randint(0, n - 1)

    for child in self.children:
      if i == child[0] and j == child[1]:
        j = random.randint(0, n - 1)
        i = random.randint(0, n - 1)
        break
      
    new_posision = self.position[:]
    new_posision[j] = i
    
    self.children.append((i, j))

    child = GameNode(position=new_posision)
    return child

In [5]:
def _calculate_cost(position):
  node = GameNode(position=position)
  node.state_cost()
  return node.cost

In [6]:
_calculate_cost(
[16, 22, 13, 18, 0, 33, 11, 29, 38, 15, 24, 21, 39, 36, 8, 3, 14, 37, 25, 6, 20, 34, 30, 35, 2, 10, 1, 26, 19, 12, 7, 32, 27, 17, 23, 4, 28, 5, 31, 9])

0

In [13]:
def solve(initial_state, C=1):
  current = initial_state
  current.state_cost()
    
  i = 0
  while True:    
    T = C / sqrt(i + 1)
    if T < 0.0005 or current.cost == 0:
      return current

    neighbor = current.generate_random_child()

    neighbor.state_cost()
    delta = neighbor.weight - current.weight

    if delta > 0:
      current = neighbor
    else:
      prob = exp(delta / T)
      print(T)
#       if i % 10 == 0:
#         print('T = %.3f | P = %.3f | D = %.3f' % (T, prob, delta), end='\n')
      if random.random() < prob:
        current = neighbor
        
    i += 1

In [14]:
def main():
  n = 40
  game_board = []

  for i in range(n):
    game_board.append([0 for i in range(n)])

  for i in range(n):
    rand_row = random.randint(0, n - 1)
    game_board[rand_row][i] = 1
    
  print('INPUT: ')
    
  _pretty_print(game_board)

  initial_state = GameNode(board=game_board)
  
  solution = solve(initial_state)
    
  print('\nSOLUTION: ')
  _pretty_print(solution.generate_board())
  
  print(solution.position)
    

if __name__  == "__main__":
    print("Starting...")
    main()

Starting...
INPUT: 
╌│╌│╌│╌│╌│╌│╌│╌│╌│╌│╌│╌│╌│╌│╌│╌│•│╌│╌│╌│╌│╌│╌│╌│╌│╌│╌│╌│╌│╌│╌│╌│╌│╌│╌│•│╌│╌│╌│╌│
╌│╌│╌│╌│╌│╌│╌│╌│╌│╌│╌│╌│╌│╌│╌│╌│╌│╌│╌│╌│╌│╌│╌│╌│╌│╌│╌│╌│╌│╌│╌│╌│╌│╌│╌│╌│╌│╌│╌│╌│
╌│╌│╌│╌│╌│╌│╌│╌│╌│╌│╌│╌│╌│╌│╌│╌│╌│╌│╌│╌│╌│╌│╌│╌│╌│╌│╌│╌│╌│╌│╌│╌│╌│╌│╌│╌│╌│╌│╌│╌│
╌│╌│╌│╌│╌│╌│╌│╌│╌│╌│╌│╌│╌│╌│╌│╌│╌│╌│╌│╌│╌│╌│╌│╌│╌│╌│╌│╌│╌│╌│•│╌│╌│╌│╌│╌│╌│╌│╌│╌│
╌│╌│╌│╌│╌│╌│╌│╌│╌│╌│╌│╌│╌│╌│╌│╌│╌│╌│╌│╌│•│╌│╌│╌│╌│╌│╌│╌│╌│╌│╌│╌│╌│╌│╌│╌│╌│╌│•│╌│
╌│•│╌│╌│╌│•│╌│╌│╌│╌│╌│•│╌│╌│╌│╌│╌│╌│╌│╌│╌│╌│╌│•│╌│╌│╌│╌│╌│╌│╌│╌│╌│╌│╌│╌│╌│╌│╌│╌│
╌│╌│╌│╌│╌│╌│╌│╌│╌│╌│╌│╌│╌│╌│╌│╌│╌│╌│╌│╌│╌│╌│╌│╌│╌│╌│╌│╌│╌│╌│╌│╌│╌│•│╌│╌│╌│╌│╌│╌│
╌│╌│╌│╌│╌│╌│╌│╌│╌│╌│╌│╌│╌│╌│╌│╌│╌│╌│╌│╌│╌│╌│╌│╌│╌│╌│╌│╌│╌│╌│╌│╌│╌│╌│╌│╌│╌│╌│╌│•│
╌│╌│╌│╌│╌│╌│╌│╌│╌│╌│╌│╌│╌│╌│╌│╌│╌│╌│╌│╌│╌│╌│╌│╌│╌│╌│╌│╌│╌│╌│╌│╌│╌│╌│╌│╌│╌│╌│╌│╌│
╌│╌│╌│╌│╌│╌│╌│╌│╌│╌│╌│╌│╌│╌│╌│╌│╌│╌│╌│╌│╌│╌│╌│╌│╌│╌│╌│╌│╌│╌│╌│╌│╌│╌│╌│╌│╌│╌│╌│╌│
╌│╌│╌│╌│╌│╌│╌│╌│╌│╌│╌│╌│╌│╌│╌│╌│╌│╌│╌│╌│╌│╌│╌│╌│•│╌│╌│╌│╌│╌│╌│╌│╌│╌│╌│╌│╌│╌│╌│╌│
╌│╌│╌│╌│•│╌│╌│╌│╌│╌│╌│╌│•│╌│╌│╌│╌│╌│•│╌│╌│╌│╌│╌│╌│╌│╌│╌│╌│╌│╌│╌│╌│╌│╌│╌│╌│╌│╌│╌│
╌│╌│╌│╌│

0.048795003647426664
0.04873701788285793
0.04867923835112355
0.048507125007266595
0.04845015831115092
0.048393391849582724
0.04828045495852676
0.0481683050859088
0.04805693313322127
0.04800153607373193
0.04789131426105757
0.047836487323493986
0.04778184825674966
0.04767312946227962
0.047565149415449405
0.04751143381455799
0.04745789978762495
0.04740454631399772
0.047298376984040214
0.0472455591261534
0.04714045207910317
0.04708816093480111
0.047036043419179864
0.04688072309384954
0.046726931351599776
0.04662524041201569
0.04657464328326223
0.046524210519923545
0.04647394123401731
0.04637388957601683
0.04632410546120795
0.046274481338274755
0.046225016352102424
0.04617570965396102
0.04612656040144425
0.04607756775840914
0.04602873089491617
0.045980048987170286
0.04593152121746254
0.045834924851410566
0.04578685464956301
0.045738935374634825
0.04569116623849538
0.04564354645876384
0.04559607525875532
0.04554875186742769
0.045454545454545456
0.045407660918649985
0.04531432544294438
0.0452

0.032651157422416956
0.03263376665824188
0.03261640365267211
0.03259906833194042
0.03258176062255373
0.0325644804512918
0.03254722774520597
0.03253000243161777
0.032495633692564134
0.032478490123081544
0.03244428422615251
0.032427221756276076
0.03241018617760822
0.03239317741958685
0.0323761954119088
0.03235924008452868
0.03234231136765754
0.0323254091917618
0.03230853348756192
0.03229168418603133
0.03227486121839514
0.03225806451612903
0.03224129401095805
0.03222454963485545
0.03220783132004154
0.03219113899898252
0.03217447260438934
0.032157832069216574
0.03214121732666125
0.03210806495339678
0.032091527190284
0.032075014954979206
0.03205852818187507
0.032042066805600074
0.03202563076101743
0.032009219983223994
0.031992834407549194
0.03197647396955397
0.03196013860502966
0.031943828249996996
0.031927542840705044
0.03191128231363014
0.03189504660547484
0.03187883565316691
0.03186264939385831
0.0318464877649241
0.031830350703961514
0.031814238148788886
0.03179815003744467
0.03178208630

0.02669753499690384
0.02668802563418119
0.026678526425610406
0.026669037353133248
0.026659558398736394
0.026650089544451302
0.026640630772354096
0.026631182064565374
0.026621743403250103
0.026612314770617474
0.026602896148920738
0.026593487520457093
0.026584088867567534
0.026574700172636696
0.02656532141809275
0.026555952586407233
0.026546593660094948
0.026537244621713762
0.02652790545386455
0.026518576139190998
0.0265092566603795
0.026499947000159
0.02649064714130088
0.026481357066618805
0.026472076758968607
0.026462806201248155
0.0264535453763972
0.026444294267397254
0.02643505285727147
0.026425821129084495
0.026416599065942358
0.026407386650992314
0.02639818386742273
0.026388990698462976
0.026379807127383248
0.026370633137494492
0.02636146871214824
0.026352313834736494
0.02634316848869161
0.026334032657486166
0.02632490632463282
0.02631578947368421
0.02630668208823282
0.02629758415191085
0.026288495648390097
0.026279416561381837
0.02627034687463669
0.02626128657194451
0.026252235637

0.02341464528954235
0.023408229439226113
0.02340181886004545
0.02339541354478655
0.0233890134862494
0.023382618677247783
0.02337622911060922
0.023369844779174954
0.023363465675799888
0.023357091793352585
0.02335072312471522
0.023344359662783535
0.02333800140046683
0.023331648330687904
0.023325300446383042
0.023318957740501974
0.023312620206007845
0.02330628783587717
0.02329996062309982
0.023293638560678977
0.023287321641631116
0.02328100985898594
0.023274703205786385
0.023268401675088573
0.023262105259961773
0.023255813953488372
0.023249527748763855
0.02324324663889676
0.023236970617008655
0.02323069967623409
0.023224433809720592
0.023218173010628604
0.023211917272131483
0.02320566658741545
0.023199420949679552
0.023193180352135665
0.023186944788008416
0.023180714250535184
0.023174488732966077
0.023168268228563865
0.023162052730603976
0.02315584223237446
0.023149636727175963
0.023143436208321693
0.023137240669137377
0.02313105010296125
0.023124864503144014
0.02311868386304882
0.0231125

...

0.005728895489202693
0.005728801479644203
0.005728707474713572
0.005728613474410421
0.00572851947873437
0.005728425487685038
0.005728331501262048
0.0057282375194650185
0.00572814354229357
0.005728049569747325
0.0057279556018259015
0.005727861638528922
0.005727767679856008
0.0057276737258067775
0.005727579776380853
0.005727485831577855
0.005727391891397405
0.005727297955839122
0.00572720402490263
0.005727110098587548
0.005727016176893498
0.0057269222598201
0.005726828347366975
0.005726734439533746
0.0057266405363200325
0.0057265466377254565
0.005726452743749639
0.005726358854392202
0.005726264969652766
0.005726171089530953
0.005726077214026385
0.005725983343138682
0.005725889476867467
0.005725795615212362
0.005725701758172987
0.005725607905748963
0.005725514057939915
0.005725420214745463
0.0057253263761652285
0.005725232542198834
0.005725138712845901
0.005725044888106052
0.005724951067978908
0.005724857252464093
0.005724763441561227
0.005724669635269932
0.0057245758335898326
0.005724482

0.005680259698321624
0.005680168062754796
0.0056800764316226994
0.005679984804924975
0.0056798931826612655
0.005679801564831212
0.00567970995143446
0.005679618342470648
0.005679526737939422
0.005679435137840422
0.005679343542173293
0.005679251950937675
0.005679160364133213
0.005679068781759548
0.005678977203816324
0.005678885630303183
0.005678794061219769
0.0056787024965657225
0.00567861093634069
0.00567851938054431
0.005678427829176229
0.005678336282236089
0.005678244739723532
0.0056781532016382035
0.005678061667979745
0.005677970138747799
0.005677878613942011
0.005677787093562022
0.005677695577607477
0.005677604066078017
0.0056775125589732895
0.005677421056292933
0.005677329558036595
0.005677238064203918
0.005677146574794544
0.005677055089808118
0.005676963609244283
0.005676872133102684
0.005676780661382962
0.005676689194084763
0.005676597731207731
0.005676506272751509
0.0056764148187157405
0.00567632336910007
0.005676231923904142
0.005676140483127599
0.005676049046770086
0.005675957

0.005638211315358279
0.005638121699741846
0.005638032088398418
0.005637942481327657
0.005637852878529223
0.005637763280002776
0.005637673685747977
0.005637584095764486
0.005637494510051965
0.005637404928610073
0.0056373153514384725
0.005637225778536822
0.005637136209904784
0.005637046645542019
0.005636957085448187
0.00563686752962295
0.005636777978065968
0.005636688430776903
0.005636598887755416
0.005636509349001167
0.005636419814513817
0.0056363302842930285
0.005636240758338461
0.005636151236649778
0.005636061719226638
0.005635972206068704
0.005635882697175637
0.005635793192547097
0.005635703692182748
0.00563561419608225
0.005635524704245264
0.005635435216671452
0.005635345733360475
0.005635256254311996
0.005635166779525675
0.005635077309001174
0.0056349878427381555
0.00563489838073628
0.005634808922995211
0.005634719469514609
0.005634630020294136
0.005634540575333454
0.005634451134632225
0.005634361698190111
0.005634272266006773
0.0056341828380818745
0.005634093414415076
0.0056340039

0.005600768926340935
0.005600681084232282
0.00560059324625663
0.005600505412413653
0.00560041758270303
0.005600329757124435
0.005600241935677545
0.005600154118362036
0.005600066305177583
0.005599978496123863
0.005599890691200551
0.005599802890407326
0.005599715093743861
0.005599627301209835
0.0055995395128049215
0.0055994517285288
0.005599363948381144
0.005599276172361631
0.005599188400469938
0.005599100632705742
0.005599012869068718
0.005598925109558542
0.005598837354174894
0.005598749602917446
0.005598661855785878
0.005598574112779866
0.005598486373899086
0.005598398639143216
0.005598310908511931
0.005598223182004909
0.005598135459621827
0.005598047741362361
0.005597960027226189
0.005597872317212988
0.005597784611322434
0.005597696909554205
0.005597609211907976
0.005597521518383428
0.005597433828980234
0.005597346143698074
0.005597258462536624
0.005597170785495563
0.005597083112574565
0.005596995443773311
0.005596907779091475
0.005596820118528737
0.0055967324620847725
0.0055966448097

0.005558558674728421
0.005558472803727826
0.005558386936706823
0.005558301073665107
0.005558215214602368
0.005558129359518302
0.0055580435084126
0.0055579576612849535
0.005557871818135057
0.005557785978962603
0.005557700143767283
0.005557614312548793
0.005557528485306823
0.005557442662041068
0.005557356842751219
0.005557271027436971
0.005557185216098015
0.005557099408734046
0.005557013605344756
0.005556927805929838
0.005556842010488986
0.005556756219021893
0.005556670431528252
0.005556584648007757
0.0055564988684601
0.0055564130928849745
0.005556327321282075
0.005556241553651096
0.0055561557899917275
0.005556070030303666
0.0055559842745866025
0.005555898522840233
0.005555812775064249
0.0055557270312583465
0.005555641291422217
0.005555555555555556
0.0055554698236580545
0.00555538409572941
0.005555298371769312
0.005555212651777458
0.005555126935753539
0.005555041223697253
0.005554955515608289
0.005554869811486344
0.005554784111331111
0.005554698415142284
0.005554612722919557
0.0055545270